In [1]:
# Required python moduldes for opening FITS files and plotting
import numpy as np
import os
import os.path as op
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from astropy.io import fits
from astropy.table import Table, hstack
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb, ZScaleInterval
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles

In [2]:
%matplotlib inline

In [3]:
version = 'v3.2' # Change to latest version.
source_table = Table.read(op.join( '/kaggle/input/hetdexescv/hetdex_sc1_v3.2.ecsv'.format(version) ) )                      

In [4]:
# Column Info
source_table.info

<Table length=232650>
     name      dtype          unit                                       description                                 n_bad 
------------- ------- ------------------- -------------------------------------------------------------------------- ------
  source_name   str26                                                                         HETDEX IAU designation      0
    source_id   int64                                                                       HETDEX Source Identifier      0
       shotid   int64                                             integer represent observation ID: int( date+obsid)      0
           RA float32                 deg                                       source_id right ascension (ICRS deg)      0
          DEC float32                 deg                                           source_id declination (ICRS deg)      0
         gmag float32                                                   sdss-g magnitude measured in HETDEX sp

I see in this data set, that there is ra and dec, and they are in the degrees.

The main focus for this dataset will be z_hetdex which is the 'spectroscopic redshift' data that 
says how relatively 'old' or far something is. The more redshift, the older the wave. 

Convert data table into pandas 

In [5]:
hetdexdf=source_table.to_pandas()

hetdexdf #we can see we converted our table into a dataframe in pandas (python's excel/sheets)

,source_name,source_id,shotid,RA,DEC,gmag,Av,z_hetdex,z_hetdex_src,z_hetdex_conf,...,lum_lya,lum_lya_err,lum_oii,lum_oii_err,flux_lya,flux_lya_err,flux_oii,flux_oii_err,sn,apcor
0,HETDEX J005604.29+000032.4,2140300000011,20181014010,14.017778,0.009023,22.096148,0.057882,-2.000,elixer,-1.0,...,NaN,NaN,9.728675e+39,1.481777e+39,NaN,NaN,9.714771,1.479659,NaN,0.921000
1,HETDEX J005604.29+000032.4,2140300000011,20180113006,14.017778,0.009023,21.276257,0.057863,-2.000,elixer,-1.0,...,NaN,NaN,7.494868e+39,1.688330e+39,NaN,NaN,7.484156,1.685917,NaN,0.936000
2,HETDEX J142914.44+510014.2,2140100002040,20190504015,217.310181,51.003937,23.528015,0.029085,-2.000,elixer,-1.0,...,NaN,NaN,4.682348e+40,7.165598e+39,NaN,NaN,7.267318,1.112149,8.05,0.941389
3,HETDEX J140222.64+510107.6,2140300000110,20180223010,210.594330,51.018787,22.087255,0.037425,-2.000,elixer,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.930000
4,HETDEX J011603.95-004509.7,2140100192799,20191224010,19.016335,-0.752552,22.246433,0.095883,-2.000,elixer,-1.0,...,NaN,NaN,2.133775e+41,7.109336e+39,NaN,NaN,38.490608,1.282434,11.90,0.915255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232645,HETDEX J110313.30+511646.4,2140100000297,20190501017,165.805222,51.279552,22.001179,0.025467,3.912,Liu+2022,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.17,0.931459
232646,HETDEX J161107.47+534208.0,2140100061927,20200527024,242.781128,53.702229,22.216864,0.036809,3.929,Liu+2022,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.57,0.878565
232647,HETDEX J125640.42+515940.8,2140000047179,20190403022,194.168396,51.994675,21.941465,0.024624,3.930,Liu+2022,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.22,0.925917
232648,HETDEX J140135.95+515711.7,2140000001451,20190504027,210.399796,51.953262,21.817234,0.035599,4.148,Liu+2022,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.05,0.938010


Now that you have the data frame for your space data, you need to import

COMAP points one , two, and three and cut out and combine the data that is from those three points. 


Tomorrow, we will also add the region stripe 82 in a seperate df. 

In [6]:
ra=hetdexdf.RA*u.deg
dec=hetdexdf.DEC*u.deg

allcoords=SkyCoord(ra, dec, frame='icrs', unit='deg')
allcoords

#COMAP overview Looking at +/-1.5 degrees around each of these three points 
COMAP_point_one = SkyCoord(ra='01h41m44.4s', dec='00d00m00s')
COMAP_point_two = SkyCoord(ra='11h20m00s', dec='52d30m00s')
COMAP_point_three = SkyCoord(ra='15h04m00s', dec='55d00m00s')

tol=1.5*u.deg


#this is going to create a array with the distances from the point for 'allcoords'
sep_point_one=allcoords.separation(COMAP_point_one)

#find the indicies of values (what rows) have data that are within the tolerance
fieldone_index=np.where(sep_point_one < tol)[0]


#this will take the specific rows that we know have data from
# the main data set and make a new dataframe with those points only
field_one_df=hetdexdf.iloc[fieldone_index]


In [7]:
sep_point_two=allcoords.separation(COMAP_point_two)
good_idx_two=np.where(sep_point_two<tol)[0]
field_two_df=hetdexdf.iloc[good_idx_two]

sep_point_three=allcoords.separation(COMAP_point_three)
good_idx_three=np.where(sep_point_three<tol)[0]
field_three_df=hetdexdf.iloc[good_idx_three]


#pd.concat (df1,df2,df3) combines several dataframes together) 
combined_df=pd.concat((field_one_df,field_two_df,field_three_df))

combined_df

,source_name,source_id,shotid,RA,DEC,gmag,Av,z_hetdex,z_hetdex_src,z_hetdex_conf,...,lum_lya,lum_lya_err,lum_oii,lum_oii_err,flux_lya,flux_lya_err,flux_oii,flux_oii_err,sn,apcor
30,HETDEX J013604.69-002806.4,2140100039333,20191222014,24.019552,-0.468454,22.367409,0.096020,-1.000000,elixer,-1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.971000
49,HETDEX J014008.26-000242.9,2140100039311,20200118009,25.034407,-0.045247,22.068237,0.084464,-1.000000,elixer,-1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.922000
99,HETDEX J014224.09+002702.3,2140100039505,20190105013,25.600368,0.450638,22.027975,0.097662,-1.000000,elixer,-1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.976000
125,HETDEX J014503.64-000345.6,2140100039569,20181011010,26.265146,-0.062678,22.087608,0.085750,-1.000000,elixer,-1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875000
141,HETDEX J014008.40-001136.8,2140100039585,20180822020,25.034983,-0.193553,22.017756,0.094533,-1.000000,elixer,-1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.932000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231520,HETDEX J150219.28+533546.2,2140100065178,20200525024,225.580338,53.596153,27.134979,0.044112,3.426397,elixer,0.098903,...,1.123051e+43,1.869950e+42,NaN,NaN,10.099308,1.681598,NaN,NaN,6.080000,0.934257
231744,HETDEX J150413.47+535415.1,2140100080782,20200425016,226.056122,53.904194,25.100616,0.039503,3.441118,elixer,0.094663,...,1.075518e+43,1.655486e+42,NaN,NaN,9.573452,1.473589,NaN,NaN,6.630000,0.919668
232015,HETDEX J150300.94+534405.7,2140000034212,20200525024,225.753937,53.734913,23.195499,0.037269,3.457100,Liu+2022,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.110001,0.934185
232188,HETDEX J150544.86+535755.7,2140100004195,20200524025,226.436920,53.965466,24.927547,0.040052,3.466329,elixer,0.099260,...,1.468493e+43,2.887168e+42,NaN,NaN,12.845826,2.525586,NaN,NaN,6.130000,0.873311


We have 11k data points that are within COMAP regions 1, 2, and 3 that are observable from this dataset. 

In [8]:
Stripedf=hetdexdf[(hetdexdf['RA']>60*u.degree)& (hetdexdf['RA']<300*u.degree) &(hetdexdf['DEC']>-1.26*u.degree) & (hetdexdf['DEC']<1.26*u.degree) ]

Stripedf


,source_name,source_id,shotid,RA,DEC,gmag,Av,z_hetdex,z_hetdex_src,z_hetdex_conf,...,lum_lya,lum_lya_err,lum_oii,lum_oii_err,flux_lya,flux_lya_err,flux_oii,flux_oii_err,sn,apcor


This dataset has no data in the stripe 82